## This file is about how to use convolutional neural network to build a classifier

In [1]:
import numpy as np
import pandas as pd
import pickle
from collections import defaultdict
import re

In [2]:
from bs4 import BeautifulSoup

In [3]:
import sys
import os

In [4]:
import keras

Using TensorFlow backend.


Tensorflow is the backend of Keras. It can also be switched to CNTK.

In [5]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical

from keras.layers import Embedding
from keras.layers import Dense,Input,Flatten
from keras.layers import Conv1D, MaxPooling1D,Dropout,Concatenate
from keras.models import Model,Sequential

## (1)Set global parameters

In [6]:
MAX_SEQUENCE_LENGTH = 1000
MAX_NB_WORDS = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2

## (2) Data Input
Data are from [IMDB Dataset](https://www.kaggle.com/c/word2vec-nlp-tutorial/data)

In [7]:
data_train = pd.read_csv('data/labeledTrainData.tsv',sep='\t')
data_train.shape

(25000, 3)

In [8]:
data_train[0:3]

,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...


## (3) Data preprocess -- remove some characters
Use BeautifulSoup to remove some html tags and remove some unwanted characters.

In [9]:
def clean_str(string):
    """
    Tokenization/string cleaning for dataset
    Every dataset is lower cased except
    """
    string = re.sub(r"\\","",string)
    string = re.sub(r"\'","",string)
    string = re.sub(r"\"","",string)
    return string.strip().lower()

texts=[]
labels=[]
for idx in range(data_train.review.shape[0]):
    text = BeautifulSoup(data_train.review[idx],'lxml')
    texts.append(clean_str(text.get_text()))
    labels.append(data_train.sentiment[idx])

## (3) Data preprocess -- data and label
Use Keras function to process the data

In [10]:
tokenizer=Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 81501 unique tokens.


Sequences is a list of list, which contains 25000 reviews. Each review is a list of its words.
Then pads each sequence to the same length.

In [11]:
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

In [12]:
labels = to_categorical(np.asarray(labels))

In [13]:
print('Shape of data tensor: ', data.shape)
print('Shape of label tensor: ',labels.shape)

Shape of data tensor:  (25000, 1000)
Shape of label tensor:  (25000, 2)


## (3) Data preprocess -- train data and validation data

In [14]:
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
num_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

x_train = data[:-num_validation_samples]
y_train = labels[:-num_validation_samples]
x_val = data[-num_validation_samples:]
y_val = labels[-num_validation_samples:]

In [15]:
print ('Number of negative and positive reviews in training and validation set')
print(y_train.sum(axis=0))
print(y_val.sum(axis=0))

Number of negative and positive reviews in training and validation set
[ 10017.   9983.]
[ 2483.  2517.]


## (4) Create CNN model

Use [Google Glove 6B vector 100d](https://nlp.stanford.edu/projects/glove/) to get the pre-trained word vector.

In [16]:
## Use pre-trained wordToVec
embeddings_index = {}
f=open('data/glove.6B/glove.6B.100d.txt',encoding='utf8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:],dtype='float32')
    embeddings_index[word]=coefs
f.close()

print('Total %s word vectors in Glove 6B 100d.' % len(embeddings_index))


embedding_matrix = np.random.random((len(word_index)+1,EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i]=embedding_vector

Total 400000 word vectors in Glove 6B 100d.


In [ ]:
model = Sequential()
model.add(Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights = [embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=True))
model.add(Conv1D(128, 5, activation='relu'))
model.add(MaxPooling1D(5))
model.add(Conv1D(128, 5, activation='relu'))
model.add(MaxPooling1D(5))
model.add(Conv1D(128, 5, activation='relu'))
model.add(MaxPooling1D(35))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 1000, 100)         8150200   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 996, 128)          64128     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 199, 128)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 195, 128)          82048     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 39, 128)           0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 35, 128)           82048     
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 1, 128)            0         
__________

## (5) Training

In [ ]:
print("--------------model fitting - convolutional 1D neural network---------------")
print("--the process of fitting is ignored here")
model.fit(x_train, y_train, validation_data=(x_val, y_val),
          epochs=10, batch_size=128)

--------------model fitting - convolutional 1D neural network---------------
--the process of fitting is ignored here
Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 401s - loss: 0.6685 - acc: 0.6230 - val_loss: 0.4998 - val_acc: 0.7814
Epoch 2/10
20000/20000 [==============================] - 473s - loss: 0.4354 - acc: 0.8068 - val_loss: 0.4885 - val_acc: 0.7550
Epoch 3/10
20000/20000 [==============================] - 421s - loss: 0.3251 - acc: 0.8659 - val_loss: 0.4391 - val_acc: 0.8080
Epoch 4/10
20000/20000 [==============================] - 414s - loss: 0.2641 - acc: 0.8930 - val_loss: 0.3371 - val_acc: 0.8618
Epoch 5/10
 7424/20000 [==========>...................] - ETA: 216s - loss: 0.2208 - acc: 0.9164

## (6) A more complex CNN model
In Yoon Kim’s paper, multiple filters have been applied.

In [ ]:
#-----------------------Complex CNN -------------------------
print ('---Start to run Complex CNN model--------------:')

embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights = [embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=True)

sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)

convs = []
filter_sizes = [3,4,5]

for fsz in filter_sizes:
    l_conv = Conv1D(filters=128,kernel_size=fsz,activation='relu')(embedded_sequences)
    l_pool = MaxPooling1D(5)(l_conv)
    convs.append(l_pool)

l_merge = Concatenate(axis=1)(convs)
l_cov1 = Conv1D(128,5,activation='relu')(l_merge)
l_pool1 = MaxPooling1D(5)(l_cov1)
l_cov2 = Conv1D(128,5,activation='relu')(l_pool1)
l_pool2 = MaxPooling1D(30)(l_cov2)
l_flat = Flatten()(l_pool2)
l_dense = Dense(128,activation='relu')(l_flat)
out = Dense(2,activation='softmax')(l_dense)

model2 = Model(sequence_input,out)
model2.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

print("model fitting - complex CNN network")
model2.summary()

In [ ]:
print("---------------model fitting - complex CNN network--------------")
print("----------The fitting process is ignored here.")
model2.fit(x_train, y_train, validation_data=(x_val, y_val),
          epochs=10, batch_size=50)